<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Gemini_Deep_Research_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import google.genai as genai
from google.genai import types
import os
import textwrap
import time

# --- 1. IMPORTS & API SETUP ---
GEMINI_API_KEY = None
try:
    # Attempt to load from Colab secrets ('GEMINI')
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback to standard environment variable
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = None

if GEMINI_API_KEY:
    try:
        # CORRECT Client initialization (explicitly passing the key)
        client = genai.Client(api_key=GEMINI_API_KEY)
        print(f"✅ Gemini client configured for **{REQUESTED_MODEL_ID}**.")
    except Exception as e:
        print(f"❌ Client initialization failed: {e}")
else:
    print("❌ API Key not found. Please ensure your key is set up.")


# --- 2. AGENT CONFIGURATIONS ---
# We keep these definitions as they influence internal deliberation quality.
def get_low_think_config():
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_level="low"
        )
    )

def get_high_think_config():
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_level="high"
        )
    )

# --- 3. CORE LOGIC (Fixed Prompt for Chain-of-Thought) ---

COMPLEX_PROMPT_FIXED = """
Analyze the following paradox and provide a resolution. You MUST structure your answer in two parts:

**Part 1: CHAIN OF REASONING**
Provide a step-by-step internal reasoning process that leads to your conclusion. Include steps for:
1.  Identifying the conflicting mandates and the severity of the conflict.
2.  Establishing the precedence based on the 'State Climate Emergency Declaration' context.
3.  Determining the necessary procedural steps (e.g., data analysis, legal action) to resolve the dilemma ethically.

**Part 2: FINAL RESOLUTION**
State the definitive action the project manager should take and the primary justification.

Paradox: The city council passed an ordinance that requires all publicly funded
construction projects to use locally sourced materials (within 50 miles)
*and* to minimize carbon emissions below the state average.
The only local quarry is 45 miles away and uses older, high-emission equipment.
The nearest low-carbon quarry is 70 miles away.
Which ordinance requirement takes precedence?
"""

def generate_and_print_results(name: str, config: types.GenerateContentConfig, prompt: str):
    """
    Calls the Gemini API with the specified configuration and prints
    the structured response.
    """
    if not client:
        return

    print(f"\n--- Running with **{name}** (Forced Chain-of-Thought) ---")

    try:
        # Call the API
        response = client.models.generate_content(
            model=REQUESTED_MODEL_ID,
            contents=prompt,
            config=config,
        )

        # 1. Print the Response
        print("\n**STRUCTURED RESPONSE:**")
        print(textwrap.indent(response.text, '    | '))

    except Exception as e:
        print(f"\n❌ An API Error occurred during the {name} run: {e}")


# --- 4. EXECUTE DEMO ---
if client:
    print("\n" + "="*80)
    print("🧠 Starting Comparative Analysis with **Forced Chain-of-Thought**")
    print("="*80)

    # Run the LOW Thinking configuration
    generate_and_print_results(
        name="LOW Thinking",
        config=get_low_think_config(),
        prompt=COMPLEX_PROMPT_FIXED
    )

    # Run the HIGH Thinking configuration
    generate_and_print_results(
        name="HIGH Thinking",
        config=get_high_think_config(),
        prompt=COMPLEX_PROMPT_FIXED
    )

✅ Gemini client configured for **gemini-3-pro-preview**.

🧠 Starting Comparative Analysis with **Forced Chain-of-Thought**

--- Running with **LOW Thinking** (Forced Chain-of-Thought) ---

**STRUCTURED RESPONSE:**
    | **Part 1: CHAIN OF REASONING**

    | 1.  **Identifying the Conflicting Mandates:** The core conflict lies between the mandate for "local sourcing" (defined as within 50 miles) and the mandate for "minimizing carbon emissions below the state average." The local quarry satisfies the geographical constraint but fails the emission constraint due to inefficient equipment. The distant quarry fails the geographical constraint but presumably satisfies the emission constraint (despite transport distance) due to cleaner operations. This is a direct contradiction where compliance with one rule necessitates violating the other.

    | 2.  **Establishing Precedence:** Context is crucial here. While both are council ordinances, the "minimizing carbon emissions" mandate likely aligns